In [88]:
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,LabelBinarizer,OneHotEncoder,StandardScaler,OrdinalEncoder
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import tensorflow as tf
from sklearn.model_selection import GridSearchCV , cross_val_score
from sklearn.metrics import  accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,roc_curve,classification_report
from termcolor import colored
import warnings
warnings.filterwarnings('ignore')
#import xgboost as xgb
#from xgboost import XGBClassifier
from sklearn.svm import SVC

import paho.mqtt.client as mqtt
import json
import csv
import os
import ast

In [89]:
def read_dataset(path):
    return pd.read_csv(path)

class DataFrameSelector (BaseEstimator, TransformerMixin):
    def __init__ (self,attribute_names):
        self.attribute_names = attribute_names
    def fit(self,X,y = None):
        return self
    def transform(self,X,y = None):
        return X[self.attribute_names].values
    
class LabelBinarizerPipeline(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, X, y=None):
        self.encoder.fit(X)
        return self
    def transform(self, X, y=None):
        return self.encoder.transform(X)

In [99]:
sub_topic = "to_model"
pub_topic = "from_model"

In [100]:
data_point = [[]]
def on_connect(client, userdata, flags, rc):
    if (rc==0):
        print ("Connection Successfull!")
        client.subscribe(sub_topic)
        print ("Subscribe to '%s'"%sub_topic)
    else:
        print("Connected with result code "+str(rc))

        
def on_message(client, userdata, msg):
    global data_point
    data_point = get_data_point(msg.payload)
    print(data_point)
    res = int(svclassifier.predict(data_point)[0])
    print("Predicted Result: ",res)
    client.publish(pub_topic,str(res))

def get_data_point(data1):
    d_point = [[]]
    try:
        data = ast.literal_eval(data1.decode("utf-8"))
        #print(data)

        for key,value in data.items():
            d_point[0].append(value)
        return d_point
        
    except Exception as e:
        print(e)

In [101]:
df_train = read_dataset('Data/wifi/wifi_data.csv')
df_test = read_dataset('Data/wifi/unknownData.csv')

In [102]:
X = df_train.iloc[:,:-1].values
y = df_train.iloc[:,-1].values

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.01,random_state = 42)

In [103]:
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_test)

print("\nConfusion Matrix:")
print(confusion_matrix(y_test,y_pred))
print("\nClassification Report:")
print(classification_report(y_test,y_pred))


Confusion Matrix:
[[3 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0]
 [0 0 3 0 0 0 0 0]
 [0 0 0 2 0 0 0 0]
 [0 0 0 0 4 0 0 0]
 [0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 4 0]
 [0 0 0 1 0 0 0 1]]

Classification Report:
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         3
         2.0       1.00      1.00      1.00         2
         3.0       1.00      1.00      1.00         3
         4.0       0.67      1.00      0.80         2
         5.0       1.00      1.00      1.00         4
         6.0       1.00      1.00      1.00         3
         7.0       1.00      1.00      1.00         4
         8.0       1.00      0.50      0.67         2

    accuracy                           0.96        23
   macro avg       0.96      0.94      0.93        23
weighted avg       0.97      0.96      0.95        23



In [104]:
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("mqtt.eclipse.org", 1883, 60)
client.loop_forever()


Connection Successfull!
Subscribe to 'to_model'
[[-88, -81, -83, -100, -71, -100, -100, -100, -100]]
Predicted Result:  7
[[-100, -81, -83, -100, -71, -100, -100, -100, -100]]
Predicted Result:  7
[[-100, -81, -83, -100, -71, -10, -50, -60, -100]]
Predicted Result:  2


KeyboardInterrupt: 